This dashboard illustrates some economic and financial aspects related to COVID-19. Economic indicators include. Financial indicators include S&P500 Index and S&P500 Index for different sectors. Furthermore, a more detailed trend is shown for the company with COVID19 vaccine. For example, the stock price for Pfizer.

Our target audience are: people who are interested in investing, who would like to get an overview on the market trend during COVID-19. Also, this visualization could help to reveal a better indication of the recovery phase after the pandemic by looking at both some economic indicators as well as relief of government policies.

In [23]:
# install yahoo finance and pandas datareader

#!pip install yfinance --upgrade --no-cache-dir
#!pip install pandas-datareader
#!pip install requests 
#!pip install beautifulsoup4 
#!pip install dash
!pip install dash_bootstrap_components

You should consider upgrading via the 'c:\users\tiger\anaconda3\python.exe -m pip install --upgrade pip' command.


In [29]:
# library import
import yfinance as yf
from pandas_datareader import data as dt
import numpy as np
import pandas as pd
import plotly.express as px
#from bs4 import BeautifulSoup
#import requests

In [37]:
# Data 1: import stock market data - S&P and sectors ETF
yf.pdr_override() 

# using different ETF to represents market indices by sector
sectors = {'SPY': 'S&P500', 'XLK': 'Information Technology', 'XLY': 'Consumer Discretionary', 'XLB': 'Materials',
           'XLC': 'Communication Services', 'XLV': 'Health Care', 'XLI': 'Industrials', 'XLP': 'Consumer Staples', 
           'XLF': 'Financial Services', 'XLU': 'Utilities', 'XLRE': 'Real Estate', 'XLE': 'Energy'}
symbols = sectors.keys()
data_source='google'
start_date = '2019-01-01'
end_date = '2020-12-31'

appended_data = []
for symbol in sectors:
    data = dt.get_data_yahoo(symbol, start_date, end_date)
    data['symbol'] = symbol
    data['sector'] = sectors[symbol]
    appended_data.append(data)
SNP = pd.concat(appended_data)
SNP.to_csv('SNP.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
# Data 2: Market shares of different sectors
# data source: https://advisor.visualcapitalist.com/sp-500-sectors-and-industries/
# the website has forbidden server access, hence the data is manually pulled and stored in csv
percent = pd.read_csv('percent_sector.csv')

# prepare the YoY price change for the sector ETF
# starting time: 2020-01-02
# ending time: end of year 2020-12-30
# drop during COVID: (min-max)/max price during 2020-01-01 - 2020-03-31
snp_soy = SNP[SNP['Date'] == '2020-01-02']
snp_eoy = SNP[SNP['Date'] == '2020-12-30']
percent = percent.merge(snp_soy[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_soy"})
percent = percent.merge(snp_eoy[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_eoy"})
percent['change_yoy'] = (percent['price_eoy']-percent['price_soy'])/percent['price_soy']

snp_covid = SNP.loc[(SNP['Date'] <= '2020-03-31') & (SNP['Date'] >= '2020-01-01')]
snp_covid_max = snp_covid.groupby(['sector'], as_index=False).max()
snp_covid_min = snp_covid.groupby(['sector'], as_index=False).min()
percent = percent.merge(snp_covid_max[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_max"})
percent = percent.merge(snp_covid_min[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_min"})
percent['drop_covid'] = (percent['price_min']-percent['price_max'])/percent['price_max']

percent = percent[['Sector', 'Percent', 'price_soy', 'price_eoy', 'change_yoy', 'price_max', 'price_min', 'drop_covid']]


#percent['change_yoy_p'] = pd.Series(["{0:.2f}%".format(val*100) for val in percent['change_yoy']], index = percent.index)
#percent['drop_covid_p'] = pd.Series(["{0:.2f}%".format(val*100) for val in percent['drop_covid']], index = percent.index)
percent

,Sector,Percent,price_soy,price_eoy,change_yoy,price_max,price_min,drop_covid
0,Information Technology,0.2748,93.389999,129.830002,0.390192,102.790001,70.400002,-0.315108
1,Health Care,0.1458,102.129997,112.260002,0.099187,104.730003,74.620003,-0.287501
2,Consumer Discretionary,0.1118,126.910004,160.690002,0.266173,132.320007,87.449997,-0.339102
3,Communication Services,0.1090,54.259998,66.940002,0.233690,57.580002,40.220001,-0.301494
4,Financial Services,0.0989,31.080000,29.120001,-0.063063,31.170000,17.660000,-0.433430
5,Industrials,0.0790,83.010002,88.139999,0.061800,85.230003,48.770000,-0.427784
6,Consumer Staples,0.0705,62.480000,66.980003,0.072023,64.790001,48.630001,-0.249421
7,Utilities,0.0313,63.810001,61.779999,-0.031813,70.980003,44.930000,-0.367005
8,Real Estate,0.0280,38.220001,36.119999,-0.054945,41.930000,25.459999,-0.392798
9,Materials,0.0256,60.700001,72.110001,0.187974,61.139999,38.349998,-0.372751


In [56]:
# Data 3: AstraZeneca (AZN), Pfizer (PFE), Johnson & Johnson (JNJ), Moderna (MRNA), Novavax (NVAX) Stock price
yf.pdr_override() 

companies = {'AZN': 'AstraZeneca', 'PFE': 'Pfizer', 'JNJ': 'Johnson & Johnson', 'MRNA': 'Moderna', 'NVAX': 'Novavax'}
vaccine_symbols = companies.keys()
data_source='google'
start_date = '2019-01-01'
end_date = '2020-12-31'

vaccine = []
for symbol in companies:
    data = dt.get_data_yahoo(symbol, start_date, end_date)
    data['symbol'] = symbol
    data['sector'] = companies[symbol]
    vaccine.append(data)
vaccine_stock = pd.concat(vaccine)
vaccine_stock.to_csv('vaccine_stock.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
# Data 4. Vaccine order data
# data source: https://launchandscalefaster.org/covid-19#Interactive%20tables%20and%20charts%20-%20COVID-19%20Vaccine%20Advance%20Market%20Commitments
# by looking at the html, found the source public Tableau dashboard and download the data from there
# Tableau view: https://public.tableau.com/views/COVID-19VaccinePurchase_16099487574570/TotalConfirmedDosesbyCountryandVaccineCandidate?:embed=y&:showVizHome=no&:host_url=https%3A%2F%2Fpublic.tableau.com%2F&:embed_code_version=3&:tabs=yes&:toolbar=yes&:animate_transition=yes&:display_static_image=no&:display_spinner=no&:display_overlay=yes&:display_count=yes&:language=en&publish=yes&:loadOrderID=1

vaccine_purchase_all = pd.read_csv('purchasing_deals.csv')
vaccine_purchase = vaccine_purchase_all[['Company','Partners','Purchaser Entity / Country', "Purchaser's country Economic  Status",
                                         'Number of Doses Procured']]

conditions = [
    (vaccine_purchase['Partners'] == 'AstraZeneca'),
    (vaccine_purchase['Company'] == 'Pfizer'),
    (vaccine_purchase['Company'] == 'Moderna'),
    (vaccine_purchase['Company'] == 'Janssen (J&J)'),
    (vaccine_purchase['Company'] == 'Novavax')
    ]

values_stock = ['AZN', 'PFE', 'MARNA', 'JNJ', 'NVAX']
values_brand = ['AstraZeneca', 'Pfizer', 'Moderna', 'Johnson & Johnson', 'Novavax']

vaccine_purchase['stock'] = np.select(conditions, values_stock, default='Other')
vaccine_purchase['brand'] = np.select(conditions, values_brand, default='Other')

vaccine_purchase = vaccine_purchase.rename(columns={'Purchaser Entity / Country': "country", 
                                                    "Purchaser's country Economic  Status": "economic status",
                                                    "Number of Doses Procured": "doses"})


vaccine_country = vaccine_purchase.groupby('country', as_index=False).sum()
vaccine_brand = vaccine_purchase.groupby('brand', as_index=False).sum()
vaccine_brand.head()

C:\Users\tiger\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tiger\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,brand,doses
0,AstraZeneca,2.768200e+09
1,Johnson & Johnson,8.780000e+08
2,Moderna,4.905000e+08
3,Novavax,1.337700e+09
4,Other,2.182000e+09


In [ ]:
# GDP for US and Canada - shaded line graph by quarter

In [ ]:
# line graphs of different sectors during COVID

In [84]:
# A tree map for different sectors with their relative change since COVID19
# line graphs with actual trend for the indices

fig = px.treemap(percent, 
                 path=['Sector'], 
                 values='Percent',
                 #color='change_yoy'
                 color='drop_covid'
                )
fig.show()

In [ ]:
# A map with pie charts for each country, where the size of the pie indicates number of vaccine ordered. separated by different companies 
# each country is shaded by income level

In [ ]:
# 1. Bar chart indicating different vaccine companies, bars = {stock price, distributed doses}
# 2. More details in time series. Line graph indicating stock price with bars indicating trading volume.

In [ ]:
# Recovery: business condidence index
# policy

In [24]:
import pandas as pd
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

In [4]:
percent = pd.read_csv('percent_sector.csv')
SNP = pd.read_csv('SNP.csv')
vaccine_stock = pd.read_csv('vaccine_stock.csv')

In [90]:
# Dash
sectors = {'SPY': 'S&P500', 'XLK': 'Information Technology', 'XLY': 'Consumer Discretionary', 'XLB': 'Materials',
           'XLC': 'Communication Services', 'XLV': 'Health Care', 'XLI': 'Industrials', 'XLP': 'Consumer Staples', 
           'XLF': 'Financial Services', 'XLU': 'Utilities', 'XLRE': 'Real Estate', 'XLE': 'Energy'}
vaccines = {'AZN': 'AstraZeneca', 'PFE': 'Pfizer', 'JNJ': 'Johnson & Johnson', 'MRNA': 'Moderna', 'NVAX': 'Novavax'}

tab2txt = "This tab shows some financial and economical indicators under the impact of COVID-19. You can monitor " \
            "some key economic indicator such as GDP, Unemployment Rate. In the next section, the stock market trend of different " \
            "sectors is displayed. Also, the treemaps illustrate the change more directly."

tab3txt = "This tab focuses on some attributes related to COVID-19 vaccines. Firstly, graphs of stock prices of vaccine companies "\
            "are plotted. Then a barchart summarizes the total number of doses ordered from each vaccine company. "\
            "Lastly, the order distribution by country is generated."


card1a = dbc.Card([
    dbc.CardBody([
        html.P("In the graph, the time for the 3 financial stimulus is displayed.\n"\
               "Phase 1: Coronavirus Preparedness and Response Supplemental Appropriations Act;\n"\
               "Phase 2: Families First Coronavirus Response Act;\n"\
               "Phase 3: CARES Act;\n"\
               "Phase 3.5: Paycheck Protection Program and Health Care Enhancement Act.",
               className="card-text"),
        html.P("Overall, roughly $2.59 trillion in new budgetary resources have been made to respond to the pandemic."\
               " It could be noticed that, the financial stimulus are released at the trough of the stock market. "\
               "After supporting the public with cash and tax reduction, the stock market has recovered and exceeded the pre-pandemic values.",
               className="card-text")
    ])
],
    color="primary",   # primary
    inverse=True,   # change color of text (black or white)
    outline=False, 
)


card1b = dbc.Card([
    dbc.CardBody([
        html.P("The graph on the left displays change in market values from 2020-01-01 to 2020-12-31 by industries, "\
               "and the graph on the right shows the change between the highest value during first quarter of 2020 and "\
               "the lowest value.",
               className="card-text"),
        html.P("It could be noticed that, with the highest proportion among all sectors, Information Technology has contributed "\
               "the greatest increase (27.5%) in stock values over the past year, where the Energy sector decreased 37%. "\
               "During pandemic, all sectors have incurred losses in value, with the most drop in Energy Sector (down by 61%), "\
               "and least impact in Consumer Staples (-25%) and Health Care (-29%).",
               className="card-text")
    ])
],
    color="primary",   # primary
    inverse=True,   # change color of text (black or white)
    outline=False, 
)


card2a = dbc.Card([
    dbc.CardBody([
        html.P("The top companies that are involved in COVID-19 vaccine development are: AstraZeneca, which co-operates with Oxford "\
               "University; Pfizer; Johnson & Johnson, which only requires one dose of injection; Moderna; and Novavax. "
               "Each vaccine has a different experiment timeline, and hence a much different stock price trend. ",
               className="card-text"),
        html.P("Some key events that might have driven the stock price is marked on the graph. It could be noticed that, "\
               "the positive news released could pull the stock price up, but only temporarily. The overall stock price also "\
               "depends on other factors of the company; for example, financial statements and other revenue streams.",
               className="card-text")
    ])
],
    color="primary",   # primary
    inverse=True,   # change color of text (black or white)
    outline=False, 
)

card2b = dbc.Card([
    dbc.CardBody([
        html.P("From the bar chart, it could be clearly viewed that AstraZeneca is current leading in the COVID-19 vaccine market, "\
               "with over 2.7B of doses ordered by different countries. And it is followed by vaccine produced by other research labs "\
               "and companies, including some in India, Russia and China. Note that the orders are mostly pre-order, meaning that "\
               "the vaccines might not be produced and ready for use yet.",
               className="card-text"),
        html.P("As shown on the map, US and India has ordered the most vaccines, followed by UK and Canada. "\
               "Except for the countries listed, also the organization, which is not displayed on the graph",
               className="card-text")
    ])
],
    color="primary",   # primary
    inverse=True,   # change color of text (black or white)
    outline=False, 
)


def get_stock(tab):
    if tab == 'finance':
        opts = [{'label': s, 'value': v} for v, s in sectors.items()]
        val = 'SPY'
    elif tab == 'vaccine':
        opts = [{'label': s, 'value': v} for v, s in vaccines.items()]
        val = 'AZN'
    return html.Div([
        dcc.Dropdown(id=tab+'_stock_dropdown', options=opts, value=val),
        dcc.Graph(id=tab+'_stock_graph')
    ])

def get_treemaps():
    return html.Div([
        html.Div(get_treemap('yoy'),
                 style={'width':'50%','display': 'inline-block',
                        'vertical-align':'top'}),
        html.Div(get_treemap('covid'),
                 style={'width':'50%','display': 'inline-block',
                        'vertical-align':'top'})
    ])

def get_treemap(s):
    if s == 'yoy':
        col = 'change_yoy'
        txt = "YoY change%"
    elif s == 'covid':
        col = 'drop_covid'
        txt = "COVID-19 impact%"
        
    fig = px.treemap(percent, path=['Sector'], values='Percent',
                     color=col, color_continuous_scale=["red","green"], range_color=[-0.8,0.8])
    fig.update_layout(title=txt,
                     margin=dict(l=10,r=10,t=60,b=60))
    
    return html.Div([
        dcc.Graph(figure=fig)
    ])

def get_map():
    fig = px.scatter_geo(vaccine_country, locations="country", locationmode = 'country names',
                         hover_name="country", size="doses", #color='lightslategray',
                         projection="natural earth")
    fig.update_layout(title="Vaccine Orders by Country",
                     margin=dict(l=10,r=20,t=60,b=60))
    fig.update_traces(marker_color='lightslategray')
    return html.Div([
        dcc.Graph(figure=fig)
    ])

def get_bar():
    fig = px.bar(vaccine_brand, x='brand', y='doses')
    fig.update_layout(title="Vaccine Orders by Company",
                     margin=dict(l=0,r=20,t=60,b=60))
    fig.update_traces(marker_color='lightslategray')
    return html.Div([
        dcc.Graph(figure=fig)
    ])


# Dash set up
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.title = 'Covid-19 Socioeconomic Dashboard'

# Base Layout
app.layout = html.Div([
    dbc.Row(html.Br(), style={'background-color':'#000000'}),
    dbc.Row([
        dbc.Col(
            html.H1(children='COVID-19 Socioeconomic Dashboard',
                    style={'textAlign': 'center', 'color':'#FFFFFF'}),md=12)],
        align="center", style={'margin':'auto', 'background-color':'#000000'}),
    dbc.Row(html.Br(), style={'background-color':'#000000'}),
    dcc.Tabs(id='dashboard-tabs', value='finance-tab',children=[
        # Tab 1: social factors
        #dcc.Tab(label='Social Factors', value='social-tab',children=[
            # div 1: number of death
        #]),
        # Tab 2: financial / economic factors
        dcc.Tab(label='Financial / Economic Factors', value='finance-tab',children=[
            # div 1: summary statistics: unemployement rate, GDP
            # div 2: stock prices line graph
            html.Br(),
            html.Div([html.P(tab2txt)], style={'width':'90%','margin':'auto'}),
            html.H3("Stock Maket Trend by Sector, 2019 Jan - 2020 Dec", style={'textAlign': 'center'}),
            dbc.Row(html.Br(), style={'background-color':'#000000'}),
            html.Div(
                dbc.Row([dbc.Col(card1a, width=3),
                         dbc.Col(get_stock('finance'), width=9)],justify="around",)
                ,style={'width':'100%','margin':'auto'}),
            # div 3: Sector Treemap
            html.Br(),
            html.H3("Treemaps by Sector, YoY% vs. COVID-19 Impact%", style={'textAlign': 'center'}),
            dbc.Row(html.Br(), style={'background-color':'#000000'}),
            html.Div(
                dbc.Row([dbc.Col(card1b, width=3),
                         dbc.Col(get_treemaps(), width=9)])
                ,style={'width':'100%','margin':'auto'})
        ]),
        # Tab 3: vaccine information
        dcc.Tab(label='Vaccine Information', value='vaccine-tab',children=[
            html.Br(),
            html.Div([html.P(tab3txt)], style={'width':'90%','margin':'auto'}),
            html.H3("Vaccine Company Stock Price, 2019 Jan - 2020 Dec", style={'textAlign': 'center'}),
            dbc.Row(html.Br(), style={'background-color':'#000000'}),
            # div 1: vaccine stock price line graphs
            html.Div(
                dbc.Row([dbc.Col(card2a, width=3),
                         dbc.Col(get_stock('vaccine'), width=9)]),
                style={'width':'100%','margin':'auto'}),
            # div 2: vaccine distribution bar
            html.Br(),
            html.H3("Vaccine Orders Summary & by Country", style={'textAlign': 'center'}),
            dbc.Row(html.Br(), style={'background-color':'#000000'}),
            html.Div(
                dbc.Row([dbc.Col(card2b, width=3),
                         dbc.Col(get_bar(), width=3),
                         dbc.Col(get_map(), width=6)]),style={'width':'100%','margin':'auto'}),
            # div 3: vaccine distribution map
        ])
    ])
])


@app.callback(
    Output('finance_stock_graph', 'figure'),
    Input('finance_stock_dropdown', 'value'))
def update_figure(stock):
    mydf = SNP
    df = mydf.loc[mydf['symbol'] == stock]

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'])
                     ])
    
    fig.update_layout(
    margin=dict(l=10,r=20,t=20,b=60),
    yaxis_title='Stock Price',
    shapes = [dict(x0='2020-03-06', x1='2020-03-06', y0=0, y1=1, xref='x', yref='paper', line_width=2),
              dict(x0='2020-03-18', x1='2020-03-18', y0=0, y1=1, xref='x', yref='paper', line_width=2),
              dict(x0='2020-03-27', x1='2020-03-27', y0=0, y1=1, xref='x', yref='paper', line_width=2),
              dict(x0='2020-04-24', x1='2020-04-24', y0=0, y1=1, xref='x', yref='paper', line_width=2)],
    annotations=[dict(x='2020-03-06', y=0.95, xref='x', yref='paper', showarrow=False, xanchor='left', text='Phase 1', textangle=-90),
                 dict(x='2020-03-18', y=0.95, xref='x', yref='paper', showarrow=False, xanchor='left', text='Phase 2', textangle=-90),
                 dict(x='2020-03-27', y=0.95, xref='x', yref='paper', showarrow=False, xanchor='left', text='Phase 3', textangle=-90),
                 dict(x='2020-04-24', y=0.95, xref='x', yref='paper', showarrow=False, xanchor='left', text='Phase 3.5', textangle=-90)]
    )

    return fig

@app.callback(
    Output('vaccine_stock_graph', 'figure'),
    Input('vaccine_stock_dropdown', 'value'))
def update_figure(stock):
    mydf = vaccine_stock
    df = mydf.loc[mydf['symbol'] == stock]

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'])
                     ])
    if stock == 'PFE':
        fig.update_layout(
            margin=dict(l=10,r=20,t=20,b=60),
            yaxis_title='Stock Price',
            shapes = [dict(x0='2020-12-08', x1='2020-12-08', y0=0, y1=1, xref='x', yref='paper', line_width=2)],
            annotations=[dict(x='2020-12-08', y=0.95, xref='x', yref='paper', showarrow=False, xanchor='left', 
                              text='Released data, 95% effective', textangle=-90)]
        )
    elif stock == 'AZN':
        fig.update_layout(
            margin=dict(l=10,r=20,t=20,b=60),
            yaxis_title='Stock Price',
            shapes = [dict(x0='2020-07-20', x1='2020-07-20', y0=0, y1=1, xref='x', yref='paper', line_width=2)],
            annotations=[dict(x='2020-07-20', y=0.95, xref='x', yref='paper', showarrow=False, xanchor='left', 
                              text='Released data, 100% effective', textangle=-90)]
        )
    elif stock == 'MRNA':
        fig.update_layout(
            margin=dict(l=10,r=20,t=20,b=60),
            yaxis_title='Stock Price',
            shapes = [dict(x0='2020-11-30', x1='2020-11-30', y0=0, y1=1, xref='x', yref='paper', line_width=2)],
            annotations=[dict(x='2020-11-30', y=0.95, xref='x', yref='paper', showarrow=False, xanchor='left', 
                              text='Released data, 94% effective', textangle=-90)]
        )
    else:
        fig.update_layout(
            margin=dict(l=10,r=20,t=20,b=60),
            yaxis_title='Stock Price')

    return fig



if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8000)

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is run